In [3]:
import pandas as pd
import numpy as np
import os
import sys
import json

In [ ]:
# define working directory
working_dir = '/Users/Sophie/Desktop/CUAHSI/CFE_Calibration'

In [4]:
# read in basin list
with open("./basin_list_516.txt", "r") as f:
    basin_list = pd.read_csv(f, header=None)


In [5]:
basin_attributes = {}

for attribute_type in ['clim', 'geol', 'hydro', 'name', 'soil', 'topo', 'vege']:
    with open("/Users/Sophie/Desktop/CUAHSI/camels_attributes_v2.0/camels_{}.txt".format(attribute_type), "r") as f:
        basin_attributes[attribute_type] = pd.read_csv(f, sep=";")
    basin_attributes[attribute_type] = basin_attributes[attribute_type].set_index("gauge_id")
    print(basin_attributes[attribute_type].loc[1022500, :])

p_mean              3.608126
pet_mean            2.119256
p_seasonality       -0.11453
frac_snow           0.245259
aridity             0.587356
high_prec_freq         20.55
high_prec_dur       1.205279
high_prec_timing         son
low_prec_freq         233.65
low_prec_dur        3.662226
low_prec_timing          jja
Name: 1022500, dtype: object
geol_1st_class                      Acid plutonic rocks
glim_1st_class_frac                            0.590658
geol_2nd_class          Siliciclastic sedimentary rocks
glim_2nd_class_frac                            0.164618
carbonate_rocks_frac                                0.0
geol_porostiy                                     0.071
geol_permeability                              -14.2138
Name: 1022500, dtype: object
q_mean              2.173062
runoff_ratio        0.602269
slope_fdc           1.776280
baseflow_index      0.554478
stream_elas         1.702782
q5                  0.204734
q95                 7.123049
high_q_freq         3.900000

In [22]:
# define atmospheric forcing file dir
#forcing_path = os.path.join(working_dir,'nldas-forcing')
forcing_path = working_dir

# define dir for exported json
config_dir = os.path.join(working_dir,'configs')

for i in range(basin_list.shape[0]): 
    if i == 1 : sys.exit(0)

    g = basin_list[0][i]
    # get forcing file
    forcing_filename = "0" + str(g) + "_hourly_nldas.csv"
    forcing_file = os.path.join(forcing_path,forcing_filename)

    # get soil params
    soil_params = {
                   "depth":2.0,             # soil depth [m]
                   "bb":4.05,               # [calibrating parameter] ‘b’ exponent on Clapp-Hornberger (1978) soil water relations
                   "mult":1000.0,           # multiplier applied to satdk to route water rapidly downslope in subsurface
                   "satdk":0.00000338,      # [calibrating parameter] saturated hydraulic conductivity [m/s]
                   "satpsi":0.355,          # saturated capillary head [m]
                   "slop":1.0,              # [calibrating parameter]
                   "smcmax":0.439,          # [calibrating parameter] saturated soil moisture content
                   "wltsmc":0.066,          # wilting point soil moisture content
                   "D":2.0                  # total thickness of the soil column [2.0 m]
                   }

    nash_storage = [0.0,0.0]
    # generate json text
    dict_json = {"forcing_file":forcing_file, 
                    "catchment_area_km2":basin_attributes['topo']['area_gages2'][g], 
                    "alpha_fc":0.33, 
                    "soil_params": soil_params, 
                    "max_gw_storage":16.0,           # [calibrating parameter]
                    "Cgw":0.01,                      # [calibrating parameter]
                    "expon":6.0,                     # [calibrating parameter]
                    "gw_storage":0.50,         
                    "soil_storage":0.667, 
                    "K_lf":0.01,                     # [calibrating parameter]
                    "K_nash":0.03,                   # [calibrating parameter]
                    "nash_storage":[0.0,0.0], 
                    "giuh_ordinates":[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1], 
                    "stand_alone":1, 
                    "unit_test":0, 
                    "compare_results_file":"",
                    }

    # save and export json files
    json_filename = 'cat_' + str(g) + '_bmi_config_cfe.json'
    json_file = os.path.join(config_dir,json_filename)
    
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(dict_json, f, ensure_ascii=False, indent=4, separators=(',', ':'))

SystemExit: 0

/Users/Sophie/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [23]:
basin_attributes['soil']

,soil_depth_pelletier,soil_depth_statsgo,soil_porosity,soil_conductivity,max_water_content,sand_frac,silt_frac,clay_frac,water_frac,organic_frac,other_frac
gauge_id,,,,,,,,,,,
1013500,7.404762,1.248408,0.461149,1.106522,0.558055,27.841827,55.156940,16.275732,5.376698,0.408717,0.000000
1022500,17.412808,1.491846,0.415905,2.375005,0.626229,59.390156,28.080937,12.037646,1.226913,0.000000,0.358472
1030500,19.011414,1.461363,0.459091,1.289807,0.653020,32.235458,51.779182,14.776824,1.634345,1.330278,0.022016
1031500,7.252557,1.279047,0.450236,1.373292,0.559123,35.269030,50.841232,12.654125,0.674594,0.000000,0.000000
1047000,5.359655,1.392779,0.422749,2.615154,0.561181,55.163133,34.185443,10.303622,0.000000,0.000000,0.147867
...,...,...,...,...,...,...,...,...,...,...,...
14309500,0.894595,0.894531,0.442804,1.335304,0.395703,37.751217,38.879406,23.213862,0.000000,0.000000,0.000000
14316700,0.879292,1.340004,0.443107,1.288301,0.584644,37.238495,38.519396,24.363634,0.000000,0.000000,0.000000
14325000,0.990318,0.892189,0.442249,1.425770,0.388650,38.961578,40.860260,20.068726,0.000000,0.000000,0.000000


In [15]:
nash_storage = "0.0,0.0"
list = [float(s) for s in nash_storage.split(',')]
list

[0.0, 0.0]

In [19]:
np.array(nash_storage)

array('0.0,0.0', dtype='<U7')